In [2]:
from gurobipy import *
nodes = ['1', '2', '3', '4']
cost = {
  ('1', '2'): 10,
  ('1', '3'): 12,
  ('1', '4'): 20,
  ('2', '1'): 10,
  ('2', '3'): 17,
  ('2', '4'): 15,
  ('3', '1'): 12,
  ('3', '2'): 17,
  ('3', '4'): 11,
  ('4', '1'): 20,
  ('4', '2'): 15,
  ('4', '3'): 11,
  }
arcs, cost = multidict(cost)
# Create optimization model
m = Model('TSP')
# Create variables
x = {}
u = {}
for i,j in arcs:
    x[i,j] = m.addVar(obj=cost[i,j], vtype = 'B',
       name='x_%s%s' % (i, j))
N = len(nodes)
for i in nodes:
    if i != nodes[N-1]:
        u[i] = m.addVar(obj=0, name='u_%s' % i)
m.update()
# Constraint for sum of incoming links to j
for j in nodes:
     m.addConstr(quicksum(x[i,j]
               for i in nodes if i != j) == 1,
                    'incom_%s' % (j))
# Constraint for sum of outgoing links from i
for i in nodes:
     m.addConstr(quicksum(x[i,j]
               for j in nodes if i != j) == 1,
                    'outgo_%s' % (i))
# Subtour elimination constraints
for i,j in arcs:
    if i != nodes[N-1] and j != nodes[N-1]:
       m.addConstr(u[i] - u[j] + N*x[i,j] <= N-1,
                'subtour_%s_%s' % (i, j))
# Compute optimal solution
m.optimize()
# Print solution
if m.status == GRB.Status.OPTIMAL:
    print('objective: %f' % m.ObjVal)
    solution = m.getAttr('x', x)
    for i,j in arcs:
         if solution[i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[i,j]))


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 15 columns and 42 nonzeros
Model fingerprint: 0xd8b068e4
Variable types: 3 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 58.0000000
Presolve time: 0.00s
Presolved: 14 rows, 15 columns, 42 nonzeros
Variable types: 3 continuous, 12 integer (12 binary)

Root relaxation: objective 4.500000e+01, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   45.00000    0    4   58.00000   45.00000  22.4%     -    0s
H    0     0                      48.0000000   45.00000 